In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/Users/manshusharma/Documents/vscode/Fraudulent Transactions'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Fontconfig warning: ignoring UTF-8: not a valid region tag
Matplotlib is building the font cache; this may take a moment.


/Users/manshusharma/Documents/vscode/Fraudulent Transactions/Fraud.csv
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/script.ipynb
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/ReadME.md
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.ipynb_checkpoints/script-checkpoint.ipynb
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/config
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/HEAD
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/description
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/index
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/COMMIT_EDITMSG
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/objects/69/38a5e4a211a218a5d45894eac8e86ac79b0a72
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/objects/c7/1a590f6c1d1d97fafd5128e7a6f5ceebe2d8b0
/Users/manshusharma/Documents/vscode/Fraudulent Transactions/.git/objects

In [6]:
df = pd.read_csv("/Users/manshusharma/Documents/vscode/Fraudulent Transactions/Fraud.csv")

In [7]:
df.shape

(6362620, 11)

In [8]:
#create a date column for time series split example
from random import randrange
import datetime as dt

def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + dt.timedelta(seconds=random_second)

df['date'] = [random_date(dt.date(2021,1,1),dt.date(2022,11,1)) for x in range(len(df))]


#used this code to generate example: https://www.folkstalk.com/2022/10/python-random-date-between-range-with-code-examples.html

In [9]:
df['date']

0          2022-03-11
1          2021-12-10
2          2021-07-14
3          2022-10-24
4          2021-06-03
              ...    
6362615    2021-03-09
6362616    2022-09-11
6362617    2021-07-18
6362618    2022-04-28
6362619    2021-03-24
Name: date, Length: 6362620, dtype: object

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier

features = [
    'amount',
    'oldbalanceOrg',
    'newbalanceOrig',
    'oldbalanceDest',
    'newbalanceDest'
]

X = df[features]
y = df['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = RandomForestClassifier()

model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(average_precision_score(y_preds, y_test))

0.6725189429547436


In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import average_precision_score

kf = KFold(n_splits=2, shuffle = True, random_state = 42)
kf.get_n_splits(X)

folds = {}
scores = []

for train, test in kf.split(X):
    # Fold
    fold_number = 1
    
    # Store fold number
    folds[fold_number] = (df.iloc[train], df.iloc[test])
    
    # Create training and testing sets
    X_train = df.iloc[train][features]
    y_train = df.iloc[train]['isFraud']
    
    X_test = df.iloc[test][features]
    y_test = df.iloc[test]['isFraud']
    
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)
    
    pr_auc = average_precision_score(y_preds, y_test)
    scores.append(pr_auc)
    
    fold_number += 1


In [14]:
np.mean(scores)

0.6641210189394728

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

scores = cross_val_score(model, X, y, scoring='accuracy', cv=kf, n_jobs=-1)

print(np.mean(scores))

0.9995663736008122


In [16]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

df = shuffle(df)
X = df[features]
y = df['isFraud']

loo = LeaveOneOut()
loo.get_n_splits(X)


all_preds = []

for train_index, test_index in loo.split(X[8000:8300]):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model = RandomForestClassifier()

    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    correct = y_preds[0] == y_test.values[0]
    
    all_preds.append(correct)
    

In [17]:
np.mean(all_preds)

1.0

In [18]:
DATE = dt.date(2022,1,1)

train_df = df[df['date'] < DATE].copy()
test_df = df[df['date'] >= DATE].copy()

X_train = train_df[features]
X_test = test_df[features]

y_train = train_df['isFraud']
y_test = test_df['isFraud']


model = RandomForestClassifier()

model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(average_precision_score(y_preds, y_test))

0.6760851656783324


In [19]:
## Jan 1 to Jan 30 2022 - Training Set, Feb 1 to Feb 30 - Testing Set
## Jan 2 to Feb 2 - Training, Feb 2 to Feb 30 - Testing Set 
## Jan 3 to Feb 3 - Training, Feb 3 to Feb 30 - Testing Set 

## Window of 30 days -> Sliding it across time 

from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(max_train_size = 50000,
                       n_splits = 5)

all_scores = []

for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = RandomForestClassifier()

    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    pr_auc = average_precision_score(y_preds, y_test)
    
    all_scores.append(pr_auc)
    
    
print(all_scores)

[0.5017047251274506, 0.5103359116671605, 0.5313079628934676, 0.5330325381533789, 0.5005675505563262]


In [20]:
## Jan 1 to Jan 30 2022 - Training Set, Feb 1 to Feb 30 - Testing Set - 30 days of training data
## Jan 1 to Feb 2 - Training, Feb 2 to Feb 30 - Testing Set - 31 days of training data
## Jan 1 to Feb 3 - Training, Feb 3 to Feb 30 - Testing Set - 31 days of training data
## Training data increases over time


from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(#max_train_size = 50000, #Remove this line for expanding window
                       n_splits = 5)

all_scores = []

for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = RandomForestClassifier()

    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    pr_auc = average_precision_score(y_preds, y_test)
    
    all_scores.append(pr_auc)
    
    
print(all_scores)

[0.6190046759777776, 0.6534806450713473, 0.6697671120322641, 0.6777104635312281, 0.6906508665865214]
